In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy import stats
from scipy.stats import linregress
import seaborn as sns
import datetime as dt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from joblib import dump, load

sns.set()

In [2]:
data = pd.read_csv("clean_bike_data.csv")
data['DATETIME'] =  pd.to_datetime(data['DATETIME'], infer_datetime_format=True)
data = data[data["YEAR"] >= 2013]
data = data.dropna()
data

,TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,LAT,LON,DATETIME
14553,Theft of Bicycle,2013,1,1,0,0,16XX W 7TH AVE,Fairview,49.265425,-123.141122,2013-01-01 00:00:00
14554,Theft of Bicycle,2013,1,1,9,0,39XX SPRINGTREE DR,Arbutus Ridge,49.250869,-123.153844,2013-01-01 09:00:00
14555,Theft of Bicycle,2013,1,1,12,0,66XX GRANVILLE ST,Kerrisdale,49.225950,-123.139915,2013-01-01 12:00:00
14556,Theft of Bicycle,2013,1,2,9,0,12XX W 14TH AVE,Fairview,49.258787,-123.133532,2013-01-02 09:00:00
14557,Theft of Bicycle,2013,1,2,12,0,10XX PACIFIC BLVD,Central Business District,49.274172,-123.115263,2013-01-02 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...
30974,Theft of Bicycle,2019,10,22,13,45,20XX W 4TH AVE,Kitsilano,49.268138,-123.151383,2019-10-22 13:45:00
30975,Theft of Bicycle,2019,10,22,18,15,27XX W BROADWAY AVE,Kitsilano,49.264101,-123.167048,2019-10-22 18:15:00
30976,Theft of Bicycle,2019,10,22,20,34,63XX YEW ST,Kerrisdale,49.228779,-123.158805,2019-10-22 20:34:00
30977,Theft of Bicycle,2019,10,23,22,48,39XX W 38TH AVE,Dunbar-Southlands,49.237564,-123.191084,2019-10-23 22:48:00


# 7. Bike Theft Prediction

In [3]:
le_neigh = preprocessing.LabelEncoder()
le_block = preprocessing.LabelEncoder()
le_weath = preprocessing.LabelEncoder()

df_class = data.copy(deep=True)
df_class["NEIGHBOURHOOD"] = df_class["NEIGHBOURHOOD"].apply(lambda x: str(x))
df_class["HUNDRED_BLOCK"] = df_class["HUNDRED_BLOCK"].apply(lambda x: str(x))
# df_class["WEATHER"] = df_class["WEATHER"]


le_neigh.fit(df_class["NEIGHBOURHOOD"])
df_class["NEIGHBOURHOOD"] = le_neigh.transform(df_class["NEIGHBOURHOOD"])

le_block.fit(df_class["HUNDRED_BLOCK"])
df_class["HUNDRED_BLOCK"] = le_block.transform(df_class["HUNDRED_BLOCK"])

# le_weath.fit(df_class["WEATHER"])
# df_class["WEATHER"] = le_weath.transform(df_class["WEATHER"])

In [4]:
X = df_class[["MONTH", "DAY", "NEIGHBOURHOOD", "HUNDRED_BLOCK"]]
y = df_class["HOUR"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.80)

In [5]:
model = DecisionTreeClassifier(max_depth=4)
model.fit(X_train, y_train)

print(model.score(X_train, y_train))
print(model.score(X_valid, y_valid))

p_test = {'max_depth':[1,2,3,4,5,6,7,8,9,10]}

tuning = GridSearchCV(estimator=DecisionTreeClassifier(),
                      param_grid = p_test,
                      scoring='accuracy',
                      n_jobs=6,
                      cv=5,
                      iid=False)

tuning.fit(X_train, y_train)
print(tuning.best_params_)

dump(model, 'DecisionTreeClassifier.joblib') 

0.09526717557251908
0.07586627995725843
{'max_depth': 1}


['DecisionTreeClassifier.joblib']

In [6]:
p_test = {'max_depth': [1,2,3,4,5,6,7,8,9,10],
         'n_estimators': [10, 20, 30, 40, 50]}

tuning = GridSearchCV(estimator=RandomForestClassifier(),
                      param_grid = p_test,
                      scoring='accuracy',
                      n_jobs=6,
                      cv=5,
                      iid=False)

tuning.fit(X_train, y_train)
print(tuning.best_params_)

model = RandomForestClassifier(n_estimators=tuning.best_params_["n_estimators"],
                               max_depth=tuning.best_params_["max_depth"])
model.fit(X_train, y_train)

print(model.score(X_train, y_train))
print(model.score(X_valid, y_valid))

dump(model, 'RandomForestClassifier.joblib') 

{'max_depth': 1, 'n_estimators': 20}
0.08458015267175573
0.08708594107769806


['RandomForestClassifier.joblib']

In [7]:
model1 = GaussianNB()
model1.fit(X_train, y_train)

print(model1.score(X_train, y_train))
print(model1.score(X_valid, y_valid))

dump(model1, 'GaussianNB.joblib') 

0.08366412213740458
0.08380399938940619


['GaussianNB.joblib']

In [8]:
model = GradientBoostingClassifier(n_estimators=40)
model.fit(X_train, y_train)

print(model.score(X_train, y_train))
print(model.score(X_valid, y_valid))

dump(model1, 'GradientBoostingClassifier(.joblib') 

0.3468702290076336
0.0792245458708594


['GradientBoostingClassifier(.joblib']